# Homework 1
## by Oscar Ligthart, Nicole Ferreira Silverio and Arend van Dormalen.

Student numbers:

Oscar Ligthart:            10557520

Nicole Ferreira Silverio : 10521933

Arend van Dormalen:        10615199


This Jupyter Notebook contains the first project for Information Retrieval 1 taught at the UvA.

# Question 1A

The chance of a type 1 error ($\alpha$) increases each time an experiment is repeated, if it's not corrected. The new $\alpha$ for _m_ experiments is $1 − (1 − \alpha)^m ≈ m\alpha$. This is called the Family-wise error rate.

# Question 1B,

The chance of a type 1 error needs to be corrected in order to find the chance of an experiment existing that rejects the null hypothesis. This can be done by the Bonferroni correction, which rejects the null hypothesis for any $p_i$ when it it lower than or equal to $\frac{\alpha}{m}$.
      

# Question 2

Assume two ranked lists created by two different rankers. List $l1$ contains documents $d1$, $d2$ and $d3$ in that order. List $l2$ contains documents $d2$, $d3$ and $d4$ in that order. Now assume that the only relevant document is $d3$, which will therefore be clicked on most often. From our judgment, it is obvious that $l2$ is the most relevant list as it has placed $d3$ on a higher position. However, in Team Draft Interleaving, these algorithms will be evaluated as having equal performance.

In this situation, $d3$ will always be the third item on the interleaved list. After the first coin flip, $d2$ will be removed from $l1$ as this document has already been supplied by $l2$.  At the second coin flip, $d3$ will be the next document for both lists. This causes the relevance for both lists to be the same, as they now both have the same chance of supplying the only relevant document to the interleaved list.

The same problem occurs when list $l1$ contains documents $d1$, $d2$, and $d3$ and list $l2$ contains documents $d2$, $d3$, and $d1$. If $d1$ and $d2$ are relevant in $48\%$ of the time and $d3$ in $4\%$ of the time, it will lead to a identical performance, while $l1$ is superior.

# Experimental Part

The following cell represents the creation of all possible relevance pairs E and P

In [ ]:
import itertools
import numpy as np
import random
import re
import matplotlib.pyplot as plt
import statistics
import scipy.stats

# first get the sequence options
relevance = ['N', 'R', 'HR']
options = list(itertools.product(relevance, repeat = 5))

# create all possible pairs in sequence options
pair_index = list(itertools.permutations(range(len(options)), 2))


pairs = []

# create nested list holding all possible pairs
for temp_pair in pair_index:
    pairs.append([options[temp_pair[0]], options[temp_pair[1]]])    


# Offline measures

Each cell represents one of the following three offline measures: Average precision, nDCG, ERR

## Average precision

In [ ]:
def get_average_precision(ranking):
    
    ''' This function calculates the average precision of a ranking of documents
    it takes a ranking as input and returns the amount of relative documents
    and the value that will represent the numerator of the equation
    '''
    
    rel = 0
    AP_numerator = 0
    # get amount of relevant documents
    for i, doc in enumerate(ranking):
        if doc == 'R' or doc == 'HR':
            rel += 1
            AP_numerator += rel/(i+1)
            
    return rel, AP_numerator

# new dict for average precision for both P and E (key is pair, value is average precisions))
AP_delta = {}

# new dict for delta measures (AP, nDCG and ERR will be stored here per pair)
delta_values = {}

# get precision for all pairs
for pair in pairs:
    # first calculate numerator for average precision for P
    P = pair[0]    
    P_rel, P_AP_numerator = get_average_precision(P)
 
    # now calculate numerator for average precision for E
    E = pair[1]
    E_rel, E_AP_numerator = get_average_precision(E)

    # get total number of relevant documents returned from query
    total_rel = P_rel + E_rel
    
    # calculate average precision for both P and E
    P_AP = P_AP_numerator/total_rel
    E_AP = E_AP_numerator/total_rel
    
    # store results in a dict
    AP_delta[(P,E)] = [E_AP - P_AP]
    
    # store AP delta measures in list format in dict
    delta_values[(P,E)] = [E_AP - P_AP]


## nDCG

In [ ]:
def get_nDCG(ranking):
    
    ''' This function calculates the normaliced discounted cumulative gain (nDCG)
    it takes a ranking of documents as input and outputs the nDCG measure value of that ranking
    '''
    
    DCG = 0
    
    # loop through ranking
    for i, rank in enumerate(ranking):
        # decide what the relative rank is
        if rank == 'HR':
            rel_r = 2
        elif rank == 'R':
            rel_r = 1
        elif rank == 'N':
            rel_r = 0
        
        DCG += (2**rel_r - 1)/(np.log2(1+(i+1)))
    
    # normalize
    DCG = DCG/len(ranking)
    
    return DCG

# new dict for average precision for both P and E (key is pair, value is average precisions))
nDCG_delta = {}

counter = 0
same_counter = 0
lower_counter = 0

# get nDCG for all pairs
for pair in pairs:    
    # first for P
    P = pair[0]
    P_DCG = get_nDCG(P)
    
    # then for E
    E = pair[1]
    E_DCG = get_nDCG(E)
    
    nDCG_delta[(P,E)] = [E_DCG - P_DCG]
    
    # add nDCG delta measure to dict
    delta_values[(P,E)].append(E_DCG - P_DCG)



## ERR 

In [ ]:
def get_ERR(ranking):
    
    ''' This function calculates the expected reciprocal rank
    it takes a ranking of documents as input and returns the ERR measure value of that rank
    this code is based on the paper on ERR
    '''
    
    # initialize variables
    ERR = 0
    p = 1
    max_rel = 2
    
    # loop through ranking
    for i, rank in enumerate(ranking):        
        # start at second rank
        if i != 0:            
            # decide what the relative rank is
            if rank == 'HR':
                rel_r = 2
            elif rank == 'R':
                rel_r = 1
            elif rank == 'N':
                rel_r = 0

            # Calculate R with the mapping function
            R = (2**rel_r - 1)/(2**max_rel)

            # Modify ERR value
            ERR += p * (R/i)

            # Modify p
            p = p*(1-R)
    
    return ERR

# new dict for ERR values for both P and E (key is pair, value is ERR value))
ERR_delta = {}

# get ERR for all pairs
for pair in pairs:
    
    # first for P
    P = pair[0]
    P_ERR = get_ERR(P)
    
    # then for E
    E = pair[1]
    E_ERR = get_ERR(E)
    
    # store delta values in dict
    ERR_delta[(P,E)] = [E_ERR - P_ERR]
    
    # add ERR delta measures to dict
    delta_values[(P,E)].append(E_ERR -  P_ERR)
    


# Filtering and interleaving pairs
In the following cells the pairs that hold delta values that are positive are extracted for further use. We extracted the pairs holding a positive delta value for all offline measures. Moreover, we took the mean of delta values from all offline measures and extracted the pairs that had a positive delta value for this mean.
Afterwards, we used Balanced Interleaving to create one ranking out of both algorithms for every pair where E outperforms P.

In [ ]:
##### Filter pairs #####

def filter_pairs(pairs_dict):
    ''' This function is used to extract pairs that have a positive delta value for the mean of all offline measures
    it takes a dictionary of pairs as input and returns a list of pairs from that dictionary that hold 
    a positive value for the delta measure
    '''
    
    pairs_list = []
    dict_items = pairs_dict.items()
    
    # loop through dictionary
    for pair in dict_items:
        # extract delta value
        scores = pair[1]
        
        # append delta value if it is above 0
        avg = sum(scores, 0.0)/len(scores)
        if avg > 0.0:
            pairs_list.append(pair[0])
    
    return pairs_list

def interleaving(pairs):
    ''' This function uses balanced interleaving to interleave a pair into one ranking
    it takes a list holding all pairs as input (where a pair represents E and P)
    and outputs a list holding the resulting ranking and a list holding the origins
    of those elements (whether the document came from E or P)
    '''
    
    # initialize output lists
    all_results = []
    all_origins = []
    
    # for all pairs
    for pair in pairs:

        # Flip a coin, assign winning and losing
        # P = pair[0], E = pair[1]
        coin_winner = random.randint(0,1)
        winner = pair[coin_winner]
        loser = pair[1 - coin_winner]

        # initiate lists
        resulting_list = []
        origin_list = []

        # iterate through lists, fill up results and origin list
        for i in range(len(winner)):
            resulting_list.append(winner[i])
            origin_list.append(coin_winner)
            resulting_list.append(loser[i])
            origin_list.append(1-coin_winner)

        all_results.append(resulting_list)
        all_origins.append(origin_list)
    
    return all_results, all_origins

In [ ]:
##### Balanced Interleaving #####

# filter and interleave pairs based on the average delta value over all offline metrics
all_pairs = filter_pairs(delta_values)
all_results, all_origins = interleaving(all_pairs)

# now get pairs for which delta measure is positive for every offline metric
AP_pairs = filter_pairs(AP_delta)
DCG_pairs = filter_pairs(nDCG_delta)
ERR_pairs = filter_pairs(ERR_delta)

# interleave the pairs for which E outperforms P per offline metric
AP_results, AP_origins = interleaving(AP_pairs)
DCG_results, DCG_origins = interleaving(DCG_pairs)
ERR_results, ERR_origins = interleaving(ERR_pairs)


# Random Click Model
The following cells represent the Random Click Model (RCM) and its simulation. Simulation is run for A) every offline measure for which E outperforms P and B) the mean delta values of all offline measures for which E outperforms P.
In this model, all queries and documents are assumed to be unique. There was a very small number of queries and documents of which the ID appeared more than once. Since we can only know the ID of a query or document, but not the content, it will be very difficult to calculate the similarity of queries or documents. Given the low occurence of duplicates it is better to treat them as unique.

In [ ]:
##### Random Click Model #####

def learn_param_RCM(data):
    ''' This function learns the probability of the event of a click based
    on a click log 
    It takes training data as input and returns rho (click probability)
    '''
    
    # open file and read
    lines=data.readlines()

    clicks = 0
    documents = 0

    # Acquire total amount of queries and clicks
    for line in lines:
        items = re.split(r'\t+',line)
        if items[2] == "Q":
            # Per query 10 documents are shown
            documents += 10
        elif items[2] == "C":
            clicks += 1
    
    # Calculate rho
    rho = clicks/documents
    
    return rho


def predict_prob_RCM(ranking, param):
    ''' Predicts a click probability '''
    
    # get the click probability for every document in ranking
    click_prob = []
    for doc in ranking:
        click_prob.append(param)
        
    return click_prob

def click_doc_RCM(click_prob):
    '''This function uses the click probability to determine whether a document is clicked on or not '''
    
    clicked = []
    for prob in click_prob:
        chance = random.random()
        if chance <= prob:
            clicked.append(1)
        else:
            clicked.append(0)
    return clicked


def RCM_simulation(pairs, origins, rho, N):
    '''The following function represents the simulation of the random click model
    
    the function takes an interleaved ranking and a list which tells which element
    in the ranking comes from which algorithm. Furthermore, parameters rho and N
    are given as input, representing click probability and number of simulations 
    respectively.
    
    output consists of a list consisting of N proportions where E
    outperforms P for all pairs
    '''
    
    # initialize a list holding the proportion of wins for E
    p_RCM_list = []
    list_wins = []
    list_trials = []

    # simulate experiment N times
    for i in range(N):
        # keep track of which algorithm won
        E_win = 0
        P_win = 0
        
        # loop through all rankings
        for j, ranking in enumerate(pairs):

            # predict probability of clicking
            click_prob = predict_prob_RCM(ranking, rho)

            # get which documents were clicked
            clicked = click_doc_RCM(click_prob)

            # now shuffle the origin list so documents are picked at random
            origin_shuffle = random.sample(origins[j], len(origins[j]))

            E_click = 0
            P_click = 0
            # check whether the clicked documents were produced by E or P
            for h, click in enumerate(clicked):
                if click == 1 and origin_shuffle[h] == 1:
                    E_click += 1
                elif click == 1 and origin_shuffle[h] == 0:
                    P_click += 1

            # determine whether E or P won
            if E_click > P_click:
                E_win += 1
            elif P_click > E_click:
                P_win += 1

        # proportion of times E won
        if (E_win - P_win) == 0:
            p = 0.5
        else:
            p = E_win / (E_win + P_win)
        p_RCM_list.append(p)
        
        list_wins.append(E_win)
        list_trials.append((E_win + P_win))
        
    return p_RCM_list, list_wins, list_trials



In [ ]:
###### Simulate random click model ######

# get parameter out of data
f=open("YandexRelPredChallenge.txt","r")
rho = learn_param_RCM(f)
f.close()

# Run N simulations
N = 50

# get the p for the average of all metrics
p_RCM_list, RCM_ALL_wins, RCM_ALL_trials = RCM_simulation(all_results, all_origins, rho, N)

# get the p for every metric 
p_AP_RCM_list, RCM_AP_wins, RCM_AP_trials = RCM_simulation(AP_results, AP_origins, rho, N)
p_DCG_RCM_list, RCM_DCG_wins, RCM_DCG_trials = RCM_simulation(DCG_results, DCG_origins, rho, N)
p_ERR_RCM_list, RCM_ERR_wins, RCM_ERR_trials = RCM_simulation(ERR_results, ERR_origins, rho, N)

print(p_RCM_list)
print(p_AP_RCM_list)
print(p_DCG_RCM_list)
print(p_ERR_RCM_list)



# Simple Dynamic Bayesian Model
The following cells represent the simple Dynamic Bayesian Model (DBM) and its simulation. Simulation is run for A) every offline measure for which E outperforms P and B) the mean delta values of all offline measures for which E outperforms P.

In order to produce a Simple Dynamic Bayesian Model based on our data, a number of assumptions were made. These assumptions were caused by data sparsity in the data set.
Firstly, like in the Random Click Model, the documents and queries are assumed to be unique.
Secondly, the sparseness of the data makes it impossible to calculate a satisfiablility score $\sigma$ for each query and document. Instead, a general $\sigma$ is calculated by finding the proportion of final clicks in a query to the total number of clicks. This method assumes that a user is satisfied after its final click in a query, meaning that any following queries in the session should be independent of this query. A general $\sigma$ based on the number of last clicks in a query was also considered. This would assume dependency between queries in a session, meaning that the results of the previous query were unsatisfiable and a new query was entered to produce more satisfiable results. The first solution was more favourable, as we have no session information in our own data.

In [ ]:
##### Simple Dynamic Bayesian Model #####


def learn_param_DBM(file):
    '''This function learns the probability of a user being satisfied after
    having clicked on a document using a click log
    it takes training data as input and returns sigma (satisfaction)
    '''
        
    lines = file.readlines()

    #previous_session = 0 # Keep track of session number to determine if click is last click.
    previous_type = ""
    
    clicks = 0
    #last_clicks_session = 0
    last_clicks_query = 0

    lines.reverse() # Reversed order, so it is detectable if a click is last.
    for line in lines:
        items = re.split(r'\t+',line) #strip tabs
        current_type = items[2]

        # check if event is a click and whether it was a last click
        if current_type == "C" and previous_type == "Q": 
            last_clicks_query += 1
        if current_type == "C":
            clicks += 1

        previous_type = current_type

    sigma = last_clicks_query/clicks
    
    return sigma
        


def predict_prob_DBM(rank, sigma):
    '''This function predicts a click probability
    for the click probability, we'll need P(A) and P(E)
    first get alpha, which will be set according to the level of relevance of a document
    '''
    
    
    # the following alpha's are based on a navigational model
    if rank == 'HR':
        alpha = 0.95
    elif rank == 'R':
        alpha = 0.5
    elif rank == 'N':
        alpha = 0.05 
    
    # check if user will click on the document (depending on alpha)
    x = random.random()
    if x <= alpha:
        P_A = 1
    else:
        P_A = 0
            
    # since we are using a simple DBM, gamma will always be one    
    gamma = 1
    
    return P_A, gamma  
       
        

def click_doc_DBM(ranking, sigma):
    '''This function decide which documents are clicked
    this function takes a ranking list and a value for the parameter sigma as input and uses
    these to determine which documents in the ranking list are clicked on
    '''
    
    
    # set P(E) to 1 (first snippet is always read)
    P_E = 1
    
    clicked = []
    
    # run through the ranking to decide whether a document will be clicked or not
    for rank in ranking:
        P_A, gamma = predict_prob_DBM(rank, sigma)
        
        # based on probability, set click to 1 or 0
        if P_A == 1 and P_E == 1:
            P_C = 1
        else:
            P_C = 0
     
        clicked.append(P_C)
        
        # if user has clicked, check if user is satisfied
        if P_C == 1:
            # now check if user is satisfied
            x = random.random()
            if x <= sigma:
                # if satisfied, user will not read any more snippets (thus click nothing)
                P_E = 0
            else:
                # if user is not satisfied, user will read next snippet (thus possibly click)
                P_E = 1 
        
    return clicked       


def DBM_simulation(pairs, origins, sigma, N):
    '''The following function represents the simulation of the dynamic bayesion model
    
    the function takes an interleaved ranking and a list which tells which element
    in the ranking comes from which algorithm. Furthermore, parameters sigma and N
    are given as input, representing satisfaction and number of simulations 
    respectively.
    
    output consists of a list consisting of N proportions where E
    outperforms P for all pairs
    '''
    
    # initiate output lists
    p_DBM_list = []
    list_wins = []
    list_trials = []
    
    # simulate experiment N times
    for i in range(N):

        # keep track of which algorithm won
        E_win = 0
        P_win = 0
        # loop through all rankings
        for j, ranking in enumerate(pairs):

            # get which documents were clicked
            clicked = click_doc_DBM(ranking, sigma)

            E_click = 0
            P_click = 0

            current_origin = origins[j]

            # check whether the clicked documents were produced by E or P
            for h, click in enumerate(clicked):
                if click == 1 and current_origin[h] == 1:
                    E_click += 1
                elif click == 1 and current_origin[h] == 0:
                    P_click += 1

            # determine whether E or P won
            if E_click > P_click:
                E_win += 1
            elif P_click > E_click:
                P_win += 1
       
        
        # proportion of times E won
        if (E_win - P_win) == 0:
            p = 0.5
        else:
            p = E_win / (E_win + P_win)
            
        p_DBM_list.append(p)
        
        list_wins.append(E_win)
        list_trials.append((E_win + P_win))
        
    return p_DBM_list, list_wins, list_trials

In [ ]:
##### Simulate dynamic bayesian model #####

# get parameter out of data
f=open("YandexRelPredChallenge.txt","r")
sigma = learn_param_DBM(f)
f.close()

# run N simulations
N = 50

# get the p for the average of all metrics
p_DBM_list, DBM_ALL_wins, DBM_ALL_trials = DBM_simulation(all_results, all_origins, sigma, N)

# get the p for every metric 
p_AP_DBM_list, DBM_AP_wins, DBM_AP_trials = DBM_simulation(AP_results, AP_origins, sigma, N)
p_DCG_DBM_list, DBM_DCG_wins, DBM_DCG_trials = DBM_simulation(DCG_results, DCG_origins, sigma, N)
p_ERR_DBM_list, DBM_ERR_wins, DBM_ERR_trials = DBM_simulation(ERR_results, ERR_origins, sigma, N)

print(p_DBM_list)
print(p_AP_DBM_list)
print(p_DCG_DBM_list)
print(p_ERR_DBM_list)

# Analysis

The following cell shows the effect of different values for delta on proportion of wins

In [ ]:
# create new dict with all values for which delta is positive
pairs_dict = {}
for key, value in delta_values.items():
    avg = sum(value, 0.0)/len(value)
    if avg > 0.0:
        pairs_dict[key] = avg

keys = sorted(pairs_dict, key=pairs_dict.get)

index = round(len(pairs_dict)/10)

##### set parameters #######

# get rho out of data
f=open("YandexRelPredChallenge.txt","r")
rho = learn_param_RCM(f)
f.close()

# get sigma out of data                     
f=open("YandexRelPredChallenge.txt","r")
sigma = learn_param_DBM(f)
f.close()

# take first 10 % of the sorted dictionary to represent the low delta values
low_deltas = keys[:index]

# take last 10 % of the sorted dictionary to represent the high delta values
high_deltas = keys[-index:]

# interleave both pair lists
low_delta_results, low_delta_origins = interleaving(low_deltas)
high_delta_results, high_delta_origins = interleaving(high_deltas)

# run the random click model on both delta's
p_RCM_ld, _,_ = RCM_simulation(low_delta_results, low_delta_origins, rho, N)
p_RCM_hd, _,_ = RCM_simulation(high_delta_results, high_delta_origins, rho, N)

# run the dynamic bayesion model on both delta's
p_DBM_ld, _,_ = DBM_simulation(low_delta_results, low_delta_origins, sigma, N)
p_DBM_hd, _,_ = DBM_simulation(high_delta_results, high_delta_origins, sigma, N)

# run statistical tests to find significant differences
scipy.stats.ttest_ind(p_DBM_ld, p_DBM_hd)

####### CREATE PLOT ########

x = [0.25,0.8]
names = ['RCM', 'DBM']

bars_lowdelta = [sum(p_RCM_ld)/len(p_RCM_ld), sum(p_DBM_ld)/len(p_DBM_ld)]
bars_highdelta = [sum(p_RCM_hd)/len(p_RCM_hd), sum(p_DBM_hd)/len(p_DBM_hd)]

ax = plt.subplot(111)
ax.bar(x[0]-0.1, bars_lowdelta[0] ,width=0.2,color='dodgerblue',align='center', label = 'low delta')
ax.bar(x[0]+0.1, bars_lowdelta[1] ,width=0.2,color='tomato',align='center', label = 'high delta')
ax.bar(x[1]-0.1, bars_highdelta[0] ,width=0.2,color='dodgerblue',align='center')
ax.bar(x[1]+0.1, bars_highdelta[1] ,width=0.2,color='tomato',align='center')
ax.set_xticks([x[0],x[1]])
ax.set_xticklabels(names)
plt.ylabel('Proportion of wins for E')
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels)
ax.set_title('Effect of delta values on click models')

plt.show()



Figure ?: Effect of delta values on click models. This figure shows the proportion of times where algorithm E provided more links that were clicked than algorithm P in a ranking. The blue bars represent 10% of the delta values where E outperforms P  just slightly (lowest delta values). The red bars represent 10% of the delta values where E outperforms P more clearly (highest delta values). The delta values where calculated by taking the mean of all results of the offline measures per pair. 

From this figure can be concluded that the models as well as the offline metrics seem to work. Pairs are extracted based on their delta measure. Pairs are only considered if E outperforms P (resulting in extraction of all positive delta values). The higher the delta value, the more E outperforms P, meaning the more relevant documents will be provided by E in comparison with P. We thus expected to find differences in only our Dynamic Bayesian Model based on these delta values. For the Random Click Model we did not expect to find any differences, since the probability of clicking on a document does not depend in any way on the relevance of a document. Thus, the manner in which E outperform P (meaning having more relevant documents in an interleaved ranking) will not have any effect on what is clicked.
For the Dynamic Bayesian Model (DBM), we did expect to find differences caused by the difference in delta values. We based this expectation on the fact that in a DBM, relevance has an impact on click probability. Therefore, algorithms that provide more relevant documents in comparison to other algorithms in a ranking will be more likely to win (provide more documents that are clicked on). This effect can be found in the figure above, which shows that within the DBM, E wins by a larger margin if it provides more relevant documents in comparison to P (thus having a larger delta value for the offline measures) . If E outperforms P just slightly (by having a low delta value), E is not more likely to provide more documents that were clicked on, since the algorithms provide roughly the same amount of relevant documents. These results confirm that our offline measures calculate their delta in such a way that it has a high value for pairs where E outperforms P by a large margin and a low value for pairs where E outperforms P by a small margin. Moreover, we can conclude that our click models seem to work based on the assumptions that we made beforehand and the results shown in this figure. The RCM is not affected by difference in relevance of provided documents, while the DBM shows that an algorithm that provides relatively more relevant documents is more likely to win (provide more documents that are clicked) than an algorithm that provides relatively less relevant documents.

In [ ]:
##### PLOT OVER THE AVERAGE OF ALL METRICS #####

# average over all metrics and standard deviation for random click model
avg_ALL_RCM = statistics.mean(p_RCM_list)
stdev_ALL_RCM = statistics.pstdev(p_RCM_list)

# average over all metrics and standard deviation for dynamic bayesian model
avg_ALL_DBM = statistics.mean(p_DBM_list)
stdev_ALL_DBM = statistics.pstdev(p_DBM_list)

# plot bar graph over all metrics, RCM compared to DBM
ax = plt.gca()

x = (1, 1.75)
y = (avg_ALL_RCM, avg_ALL_DBM)
z = (stdev_ALL_RCM, stdev_ALL_DBM)
names = ('RCM', 'DBM')
clr = ('tomato', 'dodgerblue')
for i in range(len(x)):
    plt.bar(x[i], y[i], width=0.5, color = clr[i], yerr = z[i], align="center")

ind = (1,1.75)
plt.xticks(ind, x)
plt.ylabel('Proportion of wins for E')
ax.set_xticklabels(names)

leg = ax.legend(names, loc = 'upper right')
ax.set_title('Average over all metrics for both the Random Click Model (RCM) and the Dynamic Bayesian Model')

# Get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)

# Change the location of the legend. 
xOffset = 0.25
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform = ax.transAxes)

plt.show()

The figure above merely implies that our implemented click models seem to work. We used the offline metrics to determine which pairs show that algorithm E provides more relevant documents in comparison to algorithm P. Afterwards, we used these pairs in our click model and simulated the model. 

We expect to find that within the Random Click Model (RCM), no offline metric will significantly differ from 0.5 (based on the plot showing the proportion of wins after taking the average delta of all offline metrics). Thus, the fact that E provides more relevant documents has no effect on its proportion of wins compared to P (50/50) in a RCM. This could be due to the fact that a RCM does not take the relevancy into account. 

When it comes to the Dynamic Bayesian Model (DBM), we expect to find that all metrics will significantly differ from 0.5. We base this expectation on the fact that a DBM takes relevancy into account. As we only simulate the pairs where E has more relevant documents compared to P, E is more likely to provide more documents that will be clicked on. Thus the proportion of wins for E will be significantly higher than 0.5.

## Hypothesis

We will use a binomial test to check for significant differences. Our null hypothesis is that E will not outperform P and thus have a win proportion of about 0.5, whereas our alternative hypothesis is that E does indeed outperform P and thus have a win proportion that is significantly higher than 0.5

In [ ]:
##### PLOTS FOR EACH METRIC #####

# average and standard deviation per metric for random click model
avg_AP_RCM = statistics.mean(p_AP_RCM_list)
stdev_AP_RCM = statistics.pstdev(p_AP_RCM_list)

avg_DCG_RCM = statistics.mean(p_DCG_RCM_list)
stdev_DCG_RCM = statistics.pstdev(p_DCG_RCM_list)

avg_ERR_RCM = statistics.mean(p_ERR_RCM_list)
stdev_ERR_RCM = statistics.pstdev(p_ERR_RCM_list)

# average and standard deviation per metric for random click model
avg_AP_DBM = statistics.mean(p_AP_DBM_list)
stdev_AP_DBM = statistics.pstdev(p_AP_DBM_list)

avg_DCG_DBM = statistics.mean(p_DCG_DBM_list)
stdev_DCG_DBM = statistics.pstdev(p_DCG_DBM_list)

avg_ERR_DBM = statistics.mean(p_ERR_DBM_list)
stdev_ERR_DBM = statistics.pstdev(p_ERR_DBM_list)


# plot bar graph over all metrics, RCM compared to DBM
ax = plt.gca()

#x = range(1, 7)
x = (2,2,2,4,4,4)
y = (avg_AP_RCM, avg_DCG_RCM, avg_ERR_RCM, avg_AP_DBM, avg_DCG_DBM, avg_ERR_DBM)
z = (stdev_AP_RCM, stdev_DCG_RCM, stdev_ERR_RCM, stdev_AP_DBM, stdev_DCG_DBM, stdev_ERR_DBM)
names_leg = ('AP', 'DCG', 'ERR', 'AP', 'DCG', 'ERR')
names = ('RCM', 'DBM')
clr = ('r', 'tomato', 'lightsalmon', 'b', 'dodgerblue', 'lightskyblue')
sides = (0.5, 0, -0.5, 0.5, 0, -0.5)
for i in range(len(x)):
    plt.bar(x[i]-sides[i], y[i], width=0.5, color = clr[i], yerr = z[i], align="center")

ind = (2,4)
plt.xticks(ind, x)
plt.ylabel('Proportion of wins for E')
ax.set_xticklabels(names)

leg = ax.legend(names_leg, loc = 'upper right')
ax.set_title('Average proportion of wins for both models')

# Get the bounding box of the original legend
bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)

# Change the location of the legend. 
xOffset = 0.25
bb.x0 += xOffset
bb.x1 += xOffset
leg.set_bbox_to_anchor(bb, transform = ax.transAxes)

plt.show()

Figure ?: Average proportion of wins for both models. The figure above shows the proportion of wins for E, meaning how often algorithm E provided more documents that were clicked on in comparison to algorithm P. The red bars indicate win proportions for the Random Click Model (RCM), whereas the blue bars indicate win proportions for the Dynamic Bayesian Model (DBM). Different shades of the colours represent the offline metrics used.

# Statistics

In [ ]:
##### STATISTICAL ANALYSIS #####

def calculate_pval(wins, trials):
    ''' This function calculates the p value after performing a binomial test
    it takes the amount of wins and the amount of trials from a simulation
    and performs a binomial test using these parameters
    '''
    
    success = 0
    
    for i in range(len(wins)):
        # check if the percentage of wins is greater than 50%
        if wins[i]/trials[i] > 0.5:
            success += 1

    # binomial test using the number of successful runs (above 50%)
    # and the number of runs in total to get a p-value
    pval = scipy.stats.binom_test(success, len(trials), p=0.5)
    
    return pval
    
##### Check which way we want #####

# statistical test over all metrics combined
pval_RCM_ALL = calculate_pval(RCM_ALL_wins, RCM_ALL_trials)
pval_DBM_ALL = calculate_pval(DBM_ALL_wins, DBM_ALL_trials)

# statistical test over AP
pval_RCM_AP = calculate_pval(RCM_AP_wins, RCM_AP_trials)
pval_DBM_AP = calculate_pval(DBM_AP_wins, DBM_AP_trials)

# statistical test over DCG
pval_RCM_DCG = calculate_pval(RCM_DCG_wins, RCM_DCG_trials)
pval_DBM_DCG = calculate_pval(DBM_DCG_wins, DBM_DCG_trials)

# statistical test over ERR
pval_RCM_ERR = calculate_pval(RCM_ERR_wins, RCM_ERR_trials)
pval_DBM_ERR = calculate_pval(DBM_ERR_wins, DBM_ERR_trials)

print('Binomial tests to compare whether proportion differs from 0.5:')
print('\n')
print('P-value for RCM all: ', pval_RCM_ALL)
print('P-value for DBM all: ', pval_DBM_ALL)
print('\n')
print('P-value for RCM Average Precision: ', pval_RCM_AP)
print('P-value for RCM DCG: ', pval_RCM_DCG)
print('P-value for RCM ERR: ', pval_RCM_ERR)
print('\n')
print('P-value for DBM AP: ', pval_DBM_AP)
print('P-value for DBM DCG: ', pval_DBM_DCG)
print('P-value for DBM ERR: ', pval_DBM_ERR)



Significance level: $\alpha = 0.05$

The p-values shown above are the result of running a binomial test on all combinations of online and offline metrics. The binomial test was run to test whether the win proportion deviated from 0.5, meaning whether E would outperform P. In the Random Click Model (RCM) it can be seen that none of the offline metrics leads to a significant difference from 0.5. Therefore, it can be concluded that within a RCM E does not seem to outperform P and we keep our null hypothesis. This makes sense, as the RCM does not base its click probability on relevancy of documents. The offline metrics were used to determine which pairs had more relevant documents from E than from P. As the relevancy is not taken into account in the RCM, we did not expect to find any significant differences with this test on the model.

However, when it comes to the Dynamic Bayesian Model (DBM) we find that all offline metrics show significant results. This means that within the DBM, all offline metrics show that algorithm E has a significantly higher proportion of wins compared to algorithm P. Hence, for DBM we reject our null hypothesis. Beforehand, we expected all these values to be significantly different based on the fact that DBM uses document relevance in order to calculate click probability. Thus, the model is more likely to click on a document that is labeled as being more relevant. We used the offline measures to determine which pairs show that algorithm E provided more relevant documents in comparison with algorithm P. These pairs where then used in the click model, where documents provided by E would be clicked on more just based on the fact that they are more relevant. Due to the fact that we find significant differences on the binomial test on all offline measures, we conclude that documents provided by E are clicked more frequently.

## Differences within models
For further analysis, we decided to check whether the offline metric that was used has an effect on the proportion of wins within both models. For this analysis we decided to use a t-test to find whether or not the proportion of wins differ between the use of different offline metrics within the click models.

In [ ]:
##### T-TEST #####

print('Random Click Model analysis:')
stat, pval_AP_DCG_RCM = scipy.stats.ttest_ind(p_AP_RCM_list, p_DCG_RCM_list)
print('T-test p-value between AP and DCG: ', pval_AP_DCG_RCM)

stat, pval_AP_ERR_RCM = scipy.stats.ttest_ind(p_AP_RCM_list, p_ERR_RCM_list)
print('T-test p-value between AP and ERR: ', pval_AP_ERR_RCM)

stat, pval_DCG_ERR_RCM = scipy.stats.ttest_ind(p_DCG_RCM_list, p_ERR_RCM_list)
print('T-test p-value between DCG and ERR: ', pval_DCG_ERR_RCM)
print('\n')

print('Dynamic Bayesian Model analysis:')
stat, pval_AP_DCG_DBM = scipy.stats.ttest_ind(p_AP_DBM_list, p_DCG_DBM_list)
print('T-test p-value between AP and DCG: ', pval_AP_DCG_DBM)

stat, pval_AP_ERR_DBM = scipy.stats.ttest_ind(p_AP_DBM_list, p_ERR_DBM_list)
print('T-test p-value between AP and ERR: ', pval_AP_ERR_DBM)

stat, pval_DCG_ERR_DBM = scipy.stats.ttest_ind(p_DCG_DBM_list, p_ERR_DBM_list)
print('T-test p-value between DCG and ERR: ', pval_DCG_ERR_DBM)


Significance level: $\alpha = 0.05$

As expected, we do not find any significant differences within the Random Click Model (RCM). As mentioned before, this is due to the fact that a RCM does not take relevancy of documents in a ranking into account. This is in line with the results shown in the graph concerning RCM, where differences between proportions of wins are minor.

For the Dynamic Bayesian Model (DBM), we find that every pair seems to significantly differ from one another. The p-values are extremely low, most likely due to the fact that our data has a very small standard deviation. By looking at the graph we find that DCG performs best, Average Precision (AP) comes in at second and ERR performs worst. 

In [ ]:
##### Simulate experiments on single data points #####

import numpy as np

# Get delta values for all offline metrics
deltas_AP = []
for pair in AP_pairs:
    deltas_AP.append(AP_delta[pair])
deltas_AP = np.squeeze(deltas_AP)
    
deltas_DCG = []
for pair in DCG_pairs:
    deltas_DCG.append(nDCG_delta[pair])
deltas_DCG = np.squeeze(deltas_DCG)
deltas_DCG = deltas_DCG/5 # Normalize over length of list

deltas_ERR = []    
for pair in ERR_pairs:
    deltas_ERR.append(ERR_delta[pair])
deltas_ERR = np.squeeze(deltas_ERR)
    
# get rho out of data
f=open("YandexRelPredChallenge.txt","r")
rho = learn_param_RCM(f)
f.close()

# get sigma out of data                     
f=open("YandexRelPredChallenge.txt","r")
sigma = learn_param_DBM(f)
f.close()

# Simulations for AP
AP_RCM_win_probs = []
AP_DBM_win_probs = []
for pair in AP_pairs:
    RCM_pair_wins = []
    DBM_pair_wins = []
    for i in range(50):    
        result, origin = interleaving([pair])
        p_RCM_ld, _, _ = RCM_simulation(result, origin, rho, 1)
        RCM_pair_wins.append(p_RCM_ld)
        p_DBM_ld, _,_ = DBM_simulation(result, origin, sigma, 1)
        DBM_pair_wins.append(p_DBM_ld)
    
    # Calculate proportions of wins for a single pair and add to list
    RCM_pair_wins = np.squeeze(RCM_pair_wins)
    RCM_win_prob = sum(RCM_pair_wins, 0.0)/len(RCM_pair_wins)
    AP_RCM_win_probs.append(RCM_win_prob)
    DBM_pair_wins = np.squeeze(DBM_pair_wins)
    DBM_win_prob = sum(DBM_pair_wins, 0.0)/len(DBM_pair_wins)
    AP_DBM_win_probs.append(DBM_win_prob)

# Simulations for DCG
DCG_RCM_win_probs = []
DCG_DBM_win_probs = []
for pair in DCG_pairs:
    RCM_pair_wins = []
    DBM_pair_wins = []
    for i in range(50):    
        result, origin = interleaving([pair])
        p_RCM_ld, _, _ = RCM_simulation(result, origin, rho, 1)
        RCM_pair_wins.append(p_RCM_ld)
        p_DBM_ld, _,_ = DBM_simulation(result, origin, sigma, 1)
        DBM_pair_wins.append(p_DBM_ld)
    
    # Calculate proportions of wins for a single pair and add to list
    RCM_pair_wins = np.squeeze(RCM_pair_wins)
    RCM_win_prob = sum(RCM_pair_wins, 0.0)/len(RCM_pair_wins)
    DCG_RCM_win_probs.append(RCM_win_prob)
    DBM_pair_wins = np.squeeze(DBM_pair_wins)
    DBM_win_prob = sum(DBM_pair_wins, 0.0)/len(DBM_pair_wins)
    DCG_DBM_win_probs.append(DBM_win_prob)


# Simulations for ERR
ERR_RCM_win_probs = []
ERR_DBM_win_probs = []
for pair in ERR_pairs:
    RCM_pair_wins = []
    DBM_pair_wins = []
    for i in range(50):    
        result, origin = interleaving([pair])
        p_RCM_ld, _, _ = RCM_simulation(result, origin, rho, 1)
        RCM_pair_wins.append(p_RCM_ld)
        p_DBM_ld, _,_ = DBM_simulation(result, origin, sigma, 1)
        DBM_pair_wins.append(p_DBM_ld)
    
    # Calculate proportions of wins for a single pair and add to list
    RCM_pair_wins = np.squeeze(RCM_pair_wins)
    RCM_win_prob = sum(RCM_pair_wins, 0.0)/len(RCM_pair_wins)
    ERR_RCM_win_probs.append(RCM_win_prob)
    DBM_pair_wins = np.squeeze(DBM_pair_wins)
    DBM_win_prob = sum(DBM_pair_wins, 0.0)/len(DBM_pair_wins)
    ERR_DBM_win_probs.append(DBM_win_prob)

In [ ]:
##### Create plots based on proportions per pair found above #####
import matplotlib.pyplot as plt

f, axarr = plt.subplots(2,3, figsize=(20,10))
extent = [0,1,0,1]

# RCM AP
heatmap, xedges, yedges = np.histogram2d(deltas_AP, AP_RCM_win_probs, bins=(20,20))
axarr[0,0].imshow(heatmap.T, extent=extent, origin='lower')
axarr[0,0].set_title("Average Precision on Random Click Model")

# RCM DCG
heatmap, xedges, yedges = np.histogram2d(deltas_DCG/5, DCG_RCM_win_probs, bins=(20,20))
axarr[0,1].imshow(heatmap.T, extent=extent, origin='lower')
axarr[0,1].set_title("Discounted Cumulative Gain on Random Click Model")

# RCM ERR
heatmap, xedges, yedges = np.histogram2d(deltas_ERR, ERR_RCM_win_probs, bins=(20,20))
axarr[0,2].imshow(heatmap.T, extent=extent, origin='lower')
axarr[0,2].set_title("Expected Reciprocal Rank on Random Click Model")

# DBM AP
heatmap, xedges, yedges = np.histogram2d(deltas_AP, AP_DBM_win_probs, bins=(20,20))
axarr[1,0].imshow(heatmap.T, extent=extent, origin='lower')
axarr[1,0].set_title("Average Precision on Dynamic Bayesian Model")

# DBM DCG
heatmap, xedges, yedges = np.histogram2d(deltas_DCG/5, DCG_DBM_win_probs, bins=(20,20))
axarr[1,1].imshow(heatmap.T, extent=extent, origin='lower')
axarr[1,1].set_title("Discounted Cumulative Gain on Dynamic Bayesian Model")

# DBM ERR
heatmap, xedges, yedges = np.histogram2d(deltas_ERR, ERR_DBM_win_probs, bins=(20,20))
axarr[1,2].imshow(heatmap.T, extent=extent, origin='lower')
axarr[1,2].set_title("Expected Reciprocal Rank on Dynamic Bayesian Model")

# Set attributes for all subplots
for x in range(2):
    for y in range(3):
        axarr[x,y].set_xlim([0,1])
        axarr[x,y].set_ylim([0,1])
        axarr[x,y].set_xlabel("Delta Score (offline)")
        axarr[x,y].set_ylabel("Win Proportion (online)")

plt.show()

In these heatmaps we compare the offline metrics to the online performance in both the Click Models. Yellow indicates a higher occurence of a combination of values, while dark blue indicates that there are no combinations of these values. The proportion of wins is calculated after simulating the click model for each single data point 100 times.

It becomes clear that the heatmaps of the offline metrics in the Random Click Model are very similar. This is in line with our previous results, as offline metrics do not influence this model. In this way, they differ from the results on the Dynamic Bayesian Model.
There are visible differences between the offline measures on the Dynamic Basyesian Model. DCG shows a clear correlation between the delta score and the win proportion. This correlation is less clear in Average Precision and Expected Reciprocal Rank. In this last measure, it should be noted that most delta scores are slightly higher than zero meaning there is a small difference between the two rankings.
These findings are in line with our previous results.

# Conclusion

Through our test of the evaluation measures, we find that DCG performs best on our data, AP has the second-best performance and ERR performs the worst.

A likely cause of the relatively poor performance by ERR is the assumption of independence between queries and documents (which is an assumption that we made within this experiment). ERR is an algorithm designed specifically to tackle the downsides of this assumption and is thus likely to perform better in a real-world application. 

DCG performs better than AP due to the fact that in this implementation of AP the relevance is binary and no disjunction is made between documents that are "relevant" and documents that are "highly relevant".

Finally, we have shown that an effect occurs between the delta measures and the proportion of wins on the dynamic bayesian model. This effect seems to be biggest when using the DCG measure, which is in line with what can be seen in figure 2. This effect does not seem to occur on the random click model.